In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
 

In [7]:
df=pd.read_csv('vadu-dummy-data.csv',header=1)
df.head()
unique=np.array([df['Class'].unique()])
print(unique)#found out there are three classes
df=df.drop(columns='SampleID')

[['Vata' 'Kapha' 'Pitta']]


In [11]:
# labelencoding on all since i am using random forest model
labels=LabelEncoder()
for i in df.columns:
    df[i]=labels.fit_transform(df[i])
print(df)

     Class  GENDER  F1  F2  F3  F4  F5  F6  F7  F8  ...  F123  F124  F125  \
0        2       1   1   2   0   0   2   1   0   1  ...     1     1     1   
1        0       1   2   0   0   1   0   1   0   1  ...     1     1     1   
2        2       1   1   2   0   3   2   3   1   1  ...     1     1     1   
3        2       1   1   2   0   2   2   1   1   1  ...     0     1     1   
4        0       1   2   0   0   0   0   0   0   1  ...     1     1     1   
..     ...     ...  ..  ..  ..  ..  ..  ..  ..  ..  ...   ...   ...   ...   
126      2       0   1   2   1   3   2   1   0   1  ...     0     1     1   
127      2       0   1   2   1   3   1   3   0   1  ...     1     1     1   
128      2       0   1   2   2   1   2   0   1   1  ...     1     1     1   
129      2       0   1   1   1   3   1   3   0   1  ...     1     1     1   
130      1       0   0   1   2   3   1   0   0   1  ...     1     1     0   

     F126  F127  F128  F129  F130  F131  F132  
0       0     1     1     1